In [38]:

import matplotlib as mpl
import pennylane as qml
from q_kernels import *
from exp_utils import *
from qml_utils import *
from math_utils import *
from pennylane import numpy as np

np.random.seed(1359)

In [58]:
def qaoa_kernel(x,y,qubits,weights, n_repeats = 1):
    P = projector(qubits)
    QAOAEmbedding(x,weights, wires= range(qubits))
    qml.adjoint(QAOAEmbedding)(y,weights, wires = range(qubits))
    return qml.expval(qml.Hermitian(P,wires = range(qubits)))



def qaoa_kernel_element(x, y, quantum_node, params):
    return quantum_node(x, y, len(quantum_node.device.wires), params, n_repeats= 1)


def trainable_kernel(X, params):
    n_qubits = 2
    # L = 2
    # weights = np.random.rand(L, 3, requires_grad = True)
    node = device_wrapper(n_qubits, qaoa_kernel)
    K= quantum_kernel_matrix(X, node, weights = params)
    return K


def _make_circular_data(num_sectors):
    """Generate datapoints arranged in an even circle."""
    center_indices = np.array(range(0, num_sectors))
    sector_angle = 2 * np.pi / num_sectors
    angles = (center_indices + 0.5) * sector_angle
    x = 0.7 * np.cos(angles)
    y = 0.7 * np.sin(angles)
    labels = 2 * np.remainder(np.floor_divide(angles, sector_angle), 2) - 1

    return x, y, labels


def make_double_cake_data(num_sectors):
    x1, y1, labels1 = _make_circular_data(num_sectors)
    x2, y2, labels2 = _make_circular_data(num_sectors)

    # x and y coordinates of the datapoints
    x = np.hstack([x1, 0.5 * x2])
    y = np.hstack([y1, 0.5 * y2])

    # Canonical form of dataset
    X = np.vstack([x, y]).T

    labels = np.hstack([labels1, -1 * labels2])

    # Canonical form of labels
    Y = labels.astype(int)

    return X, Y

In [76]:
import pennylane as qml

def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])
    
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)


def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

dev = qml.device("default.qubit", wires=5, shots=None)
wires = dev.wires.tolist()

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

In [101]:
init_params = random_params(num_wires=5, num_layers=2)
num_sectors = 3
#X, Y = make_double_cake_data(num_sectors)
X,Y = checkerboard(30)
kernel_value = kernel(X[0], X[1], init_params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")

init_kernel = lambda x1, x2: kernel(x1, x2, init_params)
K_init = qml.kernels.square_kernel_matrix(X, 
                    init_kernel,
                    assume_normalized_kernel=True)

with np.printoptions(precision=3, suppress=True):
  #  print(K_init)

The kernel value between the first and second datapoint is 0.962
[[1.    0.962 0.794 0.846 0.822 0.79  0.985 0.874 0.977 0.947 0.844 0.986
  0.968 0.983 0.977 0.902 0.947 0.873 0.99  0.907 0.866 0.842 0.965 0.923
  0.925 0.822 0.983 0.91  0.986 0.905]
 [0.962 1.    0.92  0.895 0.927 0.917 0.985 0.971 0.984 0.967 0.889 0.981
  0.94  0.985 0.981 0.919 0.971 0.971 0.978 0.98  0.967 0.894 0.94  0.969
  0.966 0.87  0.981 0.981 0.979 0.981]
 [0.794 0.92  1.    0.886 0.983 1.    0.867 0.982 0.879 0.845 0.875 0.837
  0.806 0.87  0.873 0.857 0.855 0.982 0.828 0.968 0.983 0.889 0.811 0.928
  0.919 0.867 0.864 0.965 0.833 0.97 ]
 [0.846 0.895 0.886 1.    0.953 0.894 0.918 0.878 0.933 0.766 1.    0.823
  0.932 0.922 0.935 0.989 0.775 0.878 0.823 0.946 0.871 1.    0.938 0.976
  0.978 0.998 0.924 0.948 0.817 0.942]
 [0.822 0.927 0.983 0.953 1.    0.987 0.9   0.963 0.914 0.824 0.945 0.842
  0.865 0.904 0.912 0.925 0.834 0.963 0.835 0.982 0.961 0.955 0.871 0.968
  0.962 0.941 0.901 0.981 0.836 0.982]


In [102]:
from sklearn.svm import SVC
svm = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)).fit(X, Y)
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)
accuracy_init = accuracy(svm, X, Y)
print(f"The accuracy of the kernel with random parameters is {accuracy_init:.3f}")



The accuracy of the kernel with random parameters is 0.533


In [103]:
kta_init = qml.kernels.target_alignment(X, Y, init_kernel, assume_normalized_kernel=True)
print(f"The kernel-target alignment for our dataset and random parameters is {kta_init:.3f}")

The kernel-target alignment for our dataset and random parameters is 0.474


In [104]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product


params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(300):
    # Choose subset of datapoints to compute the KTA on.
    subset = np.random.choice(list(range(len(X))), 7)
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

Step 10 - Alignment = 0.474
Step 20 - Alignment = 0.474
Step 30 - Alignment = 0.474
Step 40 - Alignment = 0.474


KeyboardInterrupt: 

In [105]:
final_kernel = lambda x1, x2: kernel(x1, x2, params)
K_init = qml.kernels.square_kernel_matrix(X, 
                    final_kernel,
                    assume_normalized_kernel=True)
kta_init = qml.kernels.target_alignment(X, Y, final_kernel, assume_normalized_kernel=True)
print(f"The kernel-target alignment for our dataset and random parameters is {kta_init:.3f}")


svm = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, final_kernel)).fit(X, Y)
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)
accuracy_init = accuracy(svm, X, Y)
print(f"The accuracy of the kernel with random parameters is {accuracy_init:.3f}")

The kernel-target alignment for our dataset and random parameters is 0.474
The accuracy of the kernel with random parameters is 0.533


[[0.81352422 0.91499328 0.36493956]
 [0.39432764 0.81886243 0.74041427]]
Step 1 - Alignment = 0.554
[[0.81317153 0.9150845  0.36417286]
 [0.39405961 0.81888505 0.74000122]]
Step 2 - Alignment = 0.554
[[0.81290009 0.91494969 0.36372661]
 [0.39382946 0.81884322 0.73974206]]
Step 3 - Alignment = 0.554
[[0.81269241 0.91479167 0.36342998]
 [0.39365197 0.81878929 0.73957472]]
Step 4 - Alignment = 0.554
[[0.81248095 0.91491639 0.36292261]
 [0.39348975 0.81882759 0.73930702]]
Step 5 - Alignment = 0.554
[[0.81212199 0.91491743 0.36221642]
 [0.39325676 0.81879371 0.73893257]]
Step 6 - Alignment = 0.554
[[0.81208436 0.91486126 0.36218094]
 [0.39323834 0.81876546 0.73891023]]
Step 7 - Alignment = 0.554
[[0.81223436 0.91483569 0.36247944]
 [0.39331304 0.81876994 0.73907348]]
Step 8 - Alignment = 0.554
[[0.81224921 0.91484035 0.3625024 ]
 [0.39332513 0.81877239 0.73908466]]
Step 9 - Alignment = 0.554
[[0.81224419 0.91492734 0.36244694]
 [0.39334467 0.8188019  0.73905697]]
Step 10 - Alignment = 0.554

KeyboardInterrupt: 

In [112]:
dev = qml.device('default.qubit', wires=2, shots=None)
@qml.qnode(dev)
def circuit(x1, x2, params):
    qml.templates.QAOAEmbedding(x1, params, wires=dev.wires)
    qml.adjoint(qml.templates.QAOAEmbedding)(x2, params,  wires=dev.wires)
    return qml.probs(wires=dev.wires)

init_params = np.random.rand(2,3, requires_grad = True)

kernel = lambda x1, x2: circuit(x1, x2, init_params)[0]

K_init = qml.kernels.square_kernel_matrix(X, 
                    kernel,
                    assume_normalized_kernel=True)

from sklearn.svm import SVC
svm = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, kernel)).fit(X, Y)
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)
accuracy_init = accuracy(svm, X, Y)
print(f"The accuracy of the kernel with random parameters is {accuracy_init:.3f}")

The accuracy of the kernel with random parameters is 0.533


In [114]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(300):
    # Choose subset of datapoints to compute the KTA on.
    subset = np.random.choice(list(range(len(X))), 7)
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

TypeError: <lambda>() takes 2 positional arguments but 3 were given

(20, 20)